In [41]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import torch 
from torch.utils.data import DataLoader , TensorDataset 
import torchvision
import torchvision.transforms.v2 as T
import torchmetrics

In [35]:
cov_type = fetch_covtype(random_state=42)

In [36]:
x_train_full , x_valid , y_train_full , y_valid = train_test_split(cov_type.data , cov_type.target , random_state=42 , 
                                                                   test_size=0.15)

x_train , x_test , y_train , y_test = train_test_split(x_train_full , y_train_full , random_state=42,
                                                       test_size=0.15)

In [46]:
cov_type.target_names

['Cover_Type']

In [43]:
y_train[0]

tensor(2)

In [51]:
x_train.shape

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [52]:
x_train = torch.FloatTensor(x_train_scaled)
y_train = torch.LongTensor(y_train)
x_valid = torch.FloatTensor(x_valid_scaled)
y_valid = torch.LongTensor(y_valid)
x_test = torch.FloatTensor(x_test_scaled)
y_test = torch.LongTensor(y_test)

In [53]:
train_dataset = TensorDataset(x_train , y_train)
train_loader = DataLoader(train_dataset , shuffle=True , pin_memory=True , batch_size=512 , 
                          num_workers=4)

val_dataset = TensorDataset(x_valid , y_valid)
val_loader = DataLoader(val_dataset , shuffle=True , pin_memory=True , batch_size=512 , 
                          num_workers=4)

test_dataset = TensorDataset(x_test , y_test)
test_loader = DataLoader(test_dataset , shuffle=True , pin_memory=True , batch_size=512 , 
                          num_workers=4)

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The train and eval function will use device : {device}" )


def eval(model , val_loader ,metric):
    with torch.no_grad():
        metric.reset()
        for x_val_batch , y_val_batch in val_loader:
            y_pred_val = model(x_val_batch)
            metric.update(y_pred_val , y_val_batch)
            
    return metric.compute


def train_and_eval(model , train_loader : DataLoader, valid_loader : DataLoader , 
                   criterion , metric , optimizer , n_epoch : int) -> dict:
    
    history = {'train_loss' : [],
               'train_accuracy' : [],
               'val_accuracy' : []}
    
    for epoch in range(n_epoch):
    
        metric.reset()
        model.train()
        total_loss = 0
        
        for x_batch , y_batch in train_loader:
            x_batch , y_batch = x_batch.to(device) , y_batch.to(device)
            
            #forward
            y_pred = model(x_batch)
            
            #loss
            loss = criterion(y_pred , y_batch)
            total_loss += loss.item()
            
            #optimizer
            optimizer.zero_grad()
            
            #back
            loss.backward()
            optimizer.step()
            
            metric.update(y_pred , y_batch)
                
        mean_loss = total_loss/len(train_loader)
        history['train_loss'].append(mean_loss)
        
        history['train_accuracy'].append(metric.compute.item())
        
        history['val_accuracy'].append(eval(model , val_loader , metric).item())
        
        print(f"Training loss : {history['train_loss'][-1]:.4f}  ",
              f"Training accuracy : {history['train_accuracy'][-1]:.4f}  ",
              f"val accuracy : {history['val_accuracy'][-1]:.4f}")
        
    return history
        
            
            
        

The train and eval function will use device : cuda
